# https://www.itis.gov/web_service.html

In [1]:
import requests
from difflib import SequenceMatcher

In [10]:
search = "bears"

In [9]:
requests.get(f"http://www.itis.gov/ITISWebService/jsonservice/searchByCommonName?srchKey={search}").json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [11]:
data = requests.get(f"https://www.itis.gov/ITISWebService/jsonservice/getITISTermsFromCommonName?srchKey={search}").content
data

b''

In [364]:
pruned_data = [x for x in data['itisTerms'] if x['commonNames'][0] is not None]
pruned_data

[{'author': 'Linnaeus, 1758',
  'class': 'gov.usgs.itis.itis_service.data.SvcItisTerm',
  'commonNames': ['domestic dog', 'domestic dog (feral)'],
  'nameUsage': 'invalid',
  'scientificName': 'Canis familiaris',
  'tsn': '183815'},
 {'author': 'Linnaeus, 1758',
  'class': 'gov.usgs.itis.itis_service.data.SvcItisTerm',
  'commonNames': ['domestic dog'],
  'nameUsage': 'valid',
  'scientificName': 'Canis lupus familiaris',
  'tsn': '726821'}]

In [365]:
results = [[SequenceMatcher(None, search.lower(), y.lower()).ratio() for y in x['commonNames']] for x in pruned_data]
results

[[1.0, 0.75], [1.0]]

In [366]:
max_value = float('-inf')
max_index = (0, 0)
max_indexes = []

for i, row in enumerate(results):
    for j, value in enumerate(row):
        if value > max_value:
            max_value = value
            max_index = (i, j)
            max_indexes.append(max_index)

In [367]:
max_indexes

[(0, 0)]

In [368]:
max_index

(0, 0)

In [369]:
pruned_data[max_index[0]]

{'author': 'Linnaeus, 1758',
 'class': 'gov.usgs.itis.itis_service.data.SvcItisTerm',
 'commonNames': ['domestic dog', 'domestic dog (feral)'],
 'nameUsage': 'invalid',
 'scientificName': 'Canis familiaris',
 'tsn': '183815'}

In [370]:
# scientific_data = requests.get(f"http://www.itis.gov/ITISWebService/jsonservice/getITISTermsFromScientificName?srchKey={pruned_data[max_index[0]]['scientificName']}").json()
# scientific_data

In [371]:
# General info about lion entry but no full taxonomic info
requests.get(f"http://www.itis.gov/ITISWebService/jsonservice/getFullRecordFromTSN?tsn={pruned_data[max_index[0]]['tsn']}").json()

{'acceptedNameList': {'acceptedNames': [{'acceptedName': 'Canis lupus familiaris',
    'acceptedTsn': '726821',
    'author': None,
    'class': 'gov.usgs.itis.itis_service.data.SvcAcceptedName'}],
  'class': 'gov.usgs.itis.itis_service.data.SvcAcceptedNameList',
  'tsn': '183815'},
 'class': 'gov.usgs.itis.itis_service.data.SvcFullRecord',
 'commentList': {'class': 'gov.usgs.itis.itis_service.data.SvcTaxonCommentList',
  'comments': [{'class': 'gov.usgs.itis.itis_service.data.SvcTaxonComment',
    'commentDetail': 'See comments under Canis lupus regarding nomenclatural conflicts',
    'commentId': '18243',
    'commentTimeStamp': '2007-08-15 10:57:25.0',
    'commentator': 'David Nicolson - ITIS Data Development Coordinator',
    'updateDate': '2007-08-15'}],
  'tsn': '183815'},
 'commonNameList': {'class': 'gov.usgs.itis.itis_service.data.SvcCommonNameList',
  'commonNames': [{'class': 'gov.usgs.itis.itis_service.data.SvcCommonName',
    'commonName': 'domestic dog',
    'language': 

In [372]:
# Full taxonomic Hierarchy info about searched animal
animal_data = requests.get(f"http://www.itis.gov/ITISWebService/jsonservice/getFullHierarchyFromTSN?tsn={pruned_data[max_index[0]]['tsn']}").json()
animal_data

{'author': '',
 'class': 'gov.usgs.itis.itis_service.data.SvcHierarchyRecordList',
 'hierarchyList': [{'author': 'Linnaeus, 1758',
   'class': 'gov.usgs.itis.itis_service.data.SvcHierarchyRecord',
   'parentName': '',
   'parentTsn': '',
   'rankName': 'Species',
   'taxonName': 'Canis familiaris',
   'tsn': '183815'}],
 'rankName': '',
 'sciName': '',
 'tsn': '183815'}

In [373]:
# Subspecies info about searched animal
requests.get(f"http://www.itis.gov/ITISWebService/jsonservice/getHierarchyDownFromTSN?tsn={pruned_data[max_index[0]]['tsn']}").json()

{'author': '',
 'class': 'gov.usgs.itis.itis_service.data.SvcHierarchyRecordList',
 'hierarchyList': [None],
 'rankName': '',
 'sciName': '',
 'tsn': '183815'}

In [374]:
to_save_dict = {
    'commonName': pruned_data[max_index[0]]['commonNames'][max_index[1]],
    'scientificName': pruned_data[max_index[0]]['scientificName'],
    'hierarchy': {}

}

In [375]:
already_seen = []
duplicate_ranks = {}
for x in animal_data['hierarchyList']:
    if x['rankName'] not in already_seen:
        already_seen.append(x['rankName'])
    else:
        duplicate_ranks[x['rankName']] = 0

In [376]:
duplicate_ranks

{}

In [377]:
for x in duplicate_ranks:
    to_save_dict['hierarchy'][x] = []

In [378]:
to_save_dict

{'commonName': 'domestic dog',
 'scientificName': 'Canis familiaris',
 'hierarchy': {}}

In [379]:
for x in animal_data['hierarchyList']:
    if x['rankName'] in duplicate_ranks:
        to_save_dict['hierarchy'][x['rankName']].append(x['taxonName'])
        continue
    to_save_dict['hierarchy'][x['rankName']] = x['taxonName']

In [380]:
to_save_dict

{'commonName': 'domestic dog',
 'scientificName': 'Canis familiaris',
 'hierarchy': {'Species': 'Canis familiaris'}}